# Upwork Project

### Import the neccessary libraries

In [622]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
from bs4 import BeautifulSoup
import pandas as pd
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [334]:
### Load up page

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://trailhead.salesforce.com/trailblazer-community/groups/0F9300000001sX6CAI?sort=LAST_MODIFIED_DATE_DESC&tab=members")
bs = BeautifulSoup(driver.page_source, 'html.parser')

### manaully press the show more button then check for how many candiates profile pages you have gotten

In [264]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="1ae37cf17367db46c21717934640c50f")>

In [265]:
bs = BeautifulSoup(driver.page_source, 'html.parser')

In [266]:
len(bs.find_all('a', {'href': re.compile('https://trailblazers.salesforce.com/profileView?')}))

41684

### Keep the links to each person's profile page in a dictionary and check if the numbers of people remain the same

In [ ]:
persons = {} #run this cell once and only once
# probably delete comment after you run once

In [267]:
for person in bs.find_all('a', {'href': re.compile('https://trailblazers.salesforce.com/profileView?')}):
    if persons.get(person):
        continue
    persons[person] = f"{person['href']}"

In [3]:
len(persons)

NameError: name 'persons' is not defined

### Use the dictornary keys to make a csv file to keep the link to each person's profile

In [ ]:
personsList = list(persons.keys())

In [270]:
data = {
    'persons': personsList
}

In [5]:

df = pd.DataFrame(data)

NameError: name 'data' is not defined

In [275]:
df.to_csv('output.csv', index=False)

# Read the data into python for the real work to begin

In [335]:
df2 = pd.read_csv('output.csv')

In [338]:

df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41684 entries, 0 to 41683
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   persons  41684 non-null  object
dtypes: object(1)
memory usage: 325.8+ KB


In [341]:
people = []

In [342]:
for person in df2['persons']:
    bs = BeautifulSoup(person, 'html.parser')
    if person not in people:
        people.append(bs.find('a')['href'])

In [343]:
len(people)

41684

### A bit of playing around revealed that I had duplicate data so I used a set to get rid of it

In [344]:
peopleSet = set(people)

In [345]:
len(peopleSet)

20842

# Algorithm and Test
- Create a list for each of the columns needed
- For each Person
  1. Get Person's profile page
  2. Make bs object with person's profile page
  3. With the bs object
    - getPersonName(bs)
    - getPersonTitleCompany(bs)
    - getPersonLocation(bs)
    - getPersonSML(bs)
    - getCompanyWebsite(bs)
    - getCompanySize(bs)
    - getPersonRole(bs)
  4. append all each data to it's affliated list respectfully.

- use the lists to create a Data dictionary
- use the dict to create csv file


In [346]:
peopleSetList = list(peopleSet)
len(peopleSetList)

20842

In [13]:
for person in list(peopleSet)[:10]:
    print(person)

https://trailblazers.salesforce.com/profileView?u=0053000000AJhTmAAL
https://trailblazers.salesforce.com/profileView?u=0053A00000CNAe5QAH
https://trailblazers.salesforce.com/profileView?u=0053000000AL4PJAA1
https://trailblazers.salesforce.com/profileView?u=0053000000B2D3ZAAV
https://trailblazers.salesforce.com/profileView?u=00530000009bCGGAA2
https://trailblazers.salesforce.com/profileView?u=005300000022x6vAAA
https://trailblazers.salesforce.com/profileView?u=0053A00000CNa74QAD
https://trailblazers.salesforce.com/profileView?u=0053000000ALTTCAA5
https://trailblazers.salesforce.com/profileView?u=0053000000ALCDwAAP
https://trailblazers.salesforce.com/profileView?u=0053000000AzlwnAAB


In [300]:
url = list(peopleSet)[13]

In [301]:
driver.get(url)

In [348]:
bs = BeautifulSoup(driver.page_source, 'html.parser')

In [357]:
print(bs.head.title.text.split(sep=" - "))

print(getPersonName(bs))
print(getPersonTitleCompany(bs))
print(getPersonLocation(bs))
print(getPersonSML(bs))
print(getCompanyWebsite(bs))
print(getCompanySize(bs))
print(getPersonRole(bs))


['Cyril LOUIS', 'CEO at Mavericx.ch | Salesforce MVP & Salesforce Platform Champion at Mavericx.ch', 'Geneva, CH User Group Leader', 'Trailblazer Profile']
('Cyril', 'NIL', 'LOUIS')
('CEO, Salesforce MVP & Salesforce Platform Champion', 'Mavericx.ch, Mavericx.ch')
Switzerland
https://www.linkedin.com/in/cyrillouis , https://www.twitter.com/cyril_louis , http://www.mavericx.ch

101-500 employees
Executive


# Functions

In [629]:
def getPersonName(bs):
    
    if bs == None:
        return 'NIL'

    try:
        initialData = bs.html.head.title.text.split(sep=" - ")[0]
        name = initialData.split(sep=' ')
    except AttributeError:
        return 'NIL','NIL','NIL'

    firstName = ''
    lastName = ''
    middleName = ''

    if len(name) == 3:
        firstName = name[0]
        middleName = name[1]
        lastName = name[2]
        
        
    elif len(name) == 2:
        firstName = name[0]
        middleName= 'NIL'
        lastName= name[1]
        
    return firstName, middleName, lastName

    

In [556]:
def getPersonCompany(bs):
    if bs == None:
        return 'NIL'
        
    company = 'NIL'
    for i in range(0, len(bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'}))):
        counter = bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[i].get_text()

        if counter == 'Company':
            company = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()
            if company == None: company = 'NIL'
            
    return company

In [585]:
def getPersonTitleCompany(bs):
    if bs == None:
        return 'NIL'
        
    try:
        initialData = bs.html.head.title.text.split(sep=" - ")
        initialDataCount = len(initialData)
    except AttributeError:
        return 'NIL','NIL'

    title = 'NIL'
    company = 'NIL'

    if initialDataCount == 4:
        if title == 'NIL':
            title = initialData[1]
            company = initialData[2]

    elif atExists(initialData):
        title, company = getTitleCompanyWithAt(initialData)
        

    elif onlyCompany(initialData):
        title, company = getOnlyCompany(initialData)

    elif initialDataCount == 2:
        title = 'NIL'
        company = 'NIL'
    
    
    return title,company
            

def atExists(initialData):
    if ' at ' in initialData[1]:
        return True
    return False

def getTitleCompanyWithAt(initialData):
    atCount = initialData[1].count(' at ')

    if atCount < 2:
        listOfTitleCompany = initialData[1].split(sep=' at ')
        title = listOfTitleCompany[0]
        company = listOfTitleCompany[1]

    if atCount == 2:
        listOfTitleCompany = initialData[1].split(sep=' | ')

        firstTitleCompanyList = listOfTitleCompany[0].split(' at ')
        firstTitle = firstTitleCompanyList[0]
        firstCompany = firstTitleCompanyList[1]

        secondTitleCompanyList = listOfTitleCompany[1].split(' at ')
        secondTitle = secondTitleCompanyList[0]
        secondCompany = secondTitleCompanyList[1]

        title = f'{firstTitle}, {secondTitle}'
        company = f'{firstCompany}, {secondCompany}'

    return title, company

def onlyCompany(initialData):
    if len(initialData[1].split()) == 1:
        if initialData[1].count('.com') > 0:
            return True
        elif initialData[1].count('.') == 0:
            return True
    return False

def getOnlyCompany(initialData):
    title = 'NIL'
    company = initialData[1]
    return title, company
    

In [558]:
def getPersonLocation(bs):
    if bs == None:
        return 'NIL'
        
    state = ''
    country = ''
    for i in range(0, len(bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'}))):
        counter = bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[i].get_text()

        if counter == 'Country/Region':
            country = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()
        if counter == 'State':
            state = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()

    if state != '' and country !='':
        return f"{state}, {country}"

    if state == '' and country !='':
        return f'{country}'

    if state != '' and country == '':
        return f'{state}'

    if state == '' and country == '':
        return 'NIL'


In [559]:
def getPersonSML(bs):
    if bs == None:
        return 'NIL'

    SMLCount = len(bs.find_all('a', {'class':'tds-button-social'}))
    SML = 'NIL'
    listOfSML = []

    if SMLCount > 0:
        for i in range(0, len(bs.find_all('a', {'class':'tds-button-social'}))):
            listOfSML.append(f"{bs.find_all('a', {'class':'tds-button-social'})[i]['href']}")

        SML = ' , '.join(listOfSML)

    return SML


In [603]:
def getCompanyWebsite(bs):
    if bs == None:
        return 'NIL'
        
    companyWebsite = 'NIL'
    for i in range(0, len(bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'}))):
        counter = bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[i].get_text()

        if counter == 'Company Website':
            companyWebsite = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()

    return companyWebsite
        

In [561]:
def getCompanySize(bs):
    if bs == None:
        return 'NIL'
        
    companySize = 'NIL'
    for i in range(0, len(bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'}))):
        counter = bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[i].get_text()

        if counter == 'Company Size':
            companySize = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()
            
    return companySize

In [562]:
def getPersonRole(bs):
    if bs == None:
        return 'NIL'
        
    role = 'NIL'
    for i in range(0, len(bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'}))):
        counter = bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[i].get_text()

        if counter == 'Role':
            role = bs.find_all('div', {'class': 'slds-truncate'})[i].get_text()
            if role == None: role = 'NIL'
            
    return role

In [476]:
bs.find_all('div', {'class': 'tds-text_bold tds-text-size_4 tds-font-size_bold slds-show_inline-block'})[0].get_text()

'Company'

# Result

In [361]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="ffd448a4b59d55a0041f17cfb7e6dc0b")>

In [359]:
peopleSetList = list(peopleSet)
len(peopleSetList)

20842

In [478]:
print(getPersonName(bs))
print(getPersonTitleCompany(bs))
print(getPersonCompany(bs))
print(getPersonLocation(bs))
print(getPersonSML(bs))
print(getCompanyWebsite(bs))
print(getCompanySize(bs))
print(getPersonRole(bs))

('Greg', 'NIL', 'Enderle')
('NIL', 'NIL')
Boston Scientific
Minnesota, United States
NIL
NIL
NIL
Administrator


In [614]:
firstNames = []
middleNames = []
lastNames = []
titles = []
companies = []
locations = []
socialMediaLinks = []
companyWebsites = []
companySizes = []
roles = []

In [457]:
import time
for person in peopleSetList[:2]:
    time.sleep(5)  
    driver.get(person)
    

In [607]:
driver.get(peopleSetList[13]) #6

In [608]:
bs = BeautifulSoup(driver.page_source, 'html.parser')

In [609]:

print(getPersonName(bs))
print(getPersonTitleCompany(bs))
print(getPersonCompany(bs))
print(getPersonLocation(bs))
print(getPersonSML(bs))
print(getCompanyWebsite(bs))
print(getCompanySize(bs))
print(getPersonRole(bs))


('Cyril', 'NIL', 'LOUIS')
('CEO at Mavericx.ch | Salesforce MVP & Salesforce Platform Champion at Mavericx.ch', 'Geneva, CH User Group Leader')
Mavericx.ch - Geneva, CH User Group Leader
Switzerland
https://www.linkedin.com/in/cyrillouis , https://www.twitter.com/cyril_louis , http://www.mavericx.ch
https://www.mavericx.ch
101-500 employees
Executive


In [615]:
i = 1

In [639]:
for person in peopleSetList[111:120]:
    driver.get(person)
    bs = BeautifulSoup(driver.page_source, 'html.parser')


    # get values
    firstName, middleName, lastName = getPersonName(bs)
    print(f"{i} NAME: ", f'{firstName} {lastName}')

    title, _ = getPersonTitleCompany(bs)
    print("TITLE: ", title)

    company = getPersonCompany(bs)
    print("COMPANY: ", company)

    location = getPersonLocation(bs)
    print("LOCATION: ", location)

    socialMedium = getPersonSML(bs)
    print("SOCIAL MEDIUM: ", socialMedium)

    companyWebsite = getCompanyWebsite(bs)
    print("COMPANY WEBSITE: ", companyWebsite)

    companySize = getCompanySize(bs)
    print("COMPANY SIZE: ", companySize)

    role = getPersonRole(bs)
    print("ROLE: ", role)

    #append values
    
    firstNames.append(firstName)

    
    middleNames.append(middleName)

   
    lastNames.append(lastName)

    
    titles.append(title)



    companies.append(company)

    locations.append(location)


    
    socialMediaLinks.append(socialMedium)



    companyWebsites.append(companyWebsite)


    
    companySizes.append(companySize)



    roles.append(role)
    i = i + 1
    print("Person DONE!")
    print("\n")

print("DONE!!")
    

109 NAME:  Mark Weathers
TITLE:  NIL
COMPANY:  NIL
LOCATION:  NIL
SOCIAL MEDIUM:  NIL
COMPANY WEBSITE:  NIL
COMPANY SIZE:  NIL
ROLE:  NIL
Person DONE!


110 NAME:  Karalee Slayton
TITLE:  Sr. Director, CSG Product Enablement & COE
COMPANY:  salesforce
LOCATION:  Indiana, United States
SOCIAL MEDIUM:  NIL
COMPANY WEBSITE:  NIL
COMPANY SIZE:  NIL
ROLE:  Executive
Person DONE!


111 NAME:  Roman Shovkun
TITLE:  VP of Sales & Marketing
COMPANY:  Invisible Solutions
LOCATION:  California, United States
SOCIAL MEDIUM:  NIL
COMPANY WEBSITE:  NIL
COMPANY SIZE:  NIL
ROLE:  Sales Representative / Manager
Person DONE!


112 NAME:  Emilee Schulzke
TITLE:  NIL
COMPANY:  Priority Dispatch Corp
LOCATION:  Utah, United States
SOCIAL MEDIUM:  NIL
COMPANY WEBSITE:  NIL
COMPANY SIZE:  101-500 employees
ROLE:  Service Agent / Supervisor
Person DONE!


113 NAME:  Phil Dalen
TITLE:  IT Release Manager
COMPANY:  Salesforce
LOCATION:  California, United States
SOCIAL MEDIUM:  NIL
COMPANY WEBSITE:  NIL
COMPANY

In [640]:
data = {
    'First Name': firstNames,
    'Middle Name': middleNames,
    'Last Name': lastNames,
    'Title(s)': titles,
    'Company(ies)': companies,
    'Location': locations,
    'Social Media Links': socialMediaLinks,
    'Company Website': companyWebsites,
    'Company Size': companySizes,
    'Role'  : roles,  
}

In [641]:
print(len(data['First Name']))
print(len(data['Last Name']))
print(len(data['Middle Name']))

print('\n')
print(len(data['Title(s)']))
print(len(data['Company(ies)']))


print(len(data['Location']))
print(len(data['Social Media Links']))

print(len(data['Company Website']))
print(len(data['Company Size']))
print(len(data['Role']))

117
117
117


117
117
117
117
117
117
117


In [642]:
df = pd.DataFrame(data)

In [643]:
df.to_csv("result.csv", index=False)

In [628]:
df

,First Name,Middle Name,Last Name,Title(s),Company(ies),Location,Social Media Links,Company Website,Company Size,Role
0,Muffy,NIL,Pease,Sr. Mgr. Solution Engineers,salesforce,"Texas, United States",NIL,NIL,NIL,Executive
1,Patrick,NIL,Thomas,NIL,NIL,NIL,NIL,NIL,NIL,NIL
2,Chris,NIL,Shaad,NIL,NIL,NIL,NIL,NIL,NIL,NIL
3,Greg,NIL,Enderle,NIL,Boston Scientific,"Minnesota, United States",NIL,NIL,NIL,Administrator
4,Jennifer,NIL,Gaynor,Sr. Database Coordinator,PolicyLink,"California, United States",NIL,https://www.policylink.org,21-100 employees,Administrator
...,...,...,...,...,...,...,...,...,...,...
95,Reza,NIL,Ghalamkarizadeh,Managing Director,aquilliance GmbH,Germany,"https://www.linkedin.com/in/rezajr , https://w...",https://www.aquilliance.de/,21-100 employees,Executive
96,Takuya,NIL,Kubo,NIL,Salesforce.com,Japan,NIL,NIL,NIL,Developer
97,Toni,NIL,Miller,NIL,Cloud Blend LLC,"Georgia, United States",NIL,NIL,NIL,Developer
98,Eric,NIL,Zimmerman,"Sr. Manager, Global Salesforce, CRM",AGCO Corporation,"Georgia, United States",NIL,NIL,NIL,Administrator


In [644]:
df = pd.read_csv('result.csv')

,First Name,Middle Name,Last Name,Title(s),Company(ies),Location,Social Media Links,Company Website,Company Size,Role
0,Muffy,NIL,Pease,Sr. Mgr. Solution Engineers,salesforce,"Texas, United States",NIL,NIL,NIL,Executive
1,Patrick,NIL,Thomas,NIL,NIL,NIL,NIL,NIL,NIL,NIL
2,Chris,NIL,Shaad,NIL,NIL,NIL,NIL,NIL,NIL,NIL
3,Greg,NIL,Enderle,NIL,Boston Scientific,"Minnesota, United States",NIL,NIL,NIL,Administrator
4,Jennifer,NIL,Gaynor,Sr. Database Coordinator,PolicyLink,"California, United States",NIL,https://www.policylink.org,21-100 employees,Administrator
...,...,...,...,...,...,...,...,...,...,...
108,Phil,Van,Dalen,IT Release Manager,Salesforce,"California, United States",NIL,NIL,NIL,Developer
109,Jennifer,NIL,Pierson,NIL,Textron Aviation,"Kentucky, United States",NIL,NIL,NIL,Developer
110,Stephanie,NIL,Carlson,Contracts Manager,InEight Inc.,"Arizona, United States",NIL,NIL,NIL,Administrator
111,Westley,NIL,Anselmo,NIL,JPD Financial,"California, United States",NIL,NIL,NIL,Developer
